In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [4]:
train_dataset = torchvision.datasets.MNIST(root='C:\\Users\\venka\\Desktop\\Pytorch\\LearningPytorch\\MNIST\\',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=False)

test_dataset = torchvision.datasets.MNIST(root='C:\\Users\\venka\\Desktop\\Pytorch\\LearningPytorch\\MNIST\\',
                                          train=False, 
                                          transform=transforms.ToTensor())

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


In [6]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [7]:
model  = ConvNet(num_classes).to(device)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [9]:
total_step = len(train_loader)

In [10]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1490
Epoch [1/5], Step [200/600], Loss: 0.1349
Epoch [1/5], Step [300/600], Loss: 0.0824
Epoch [1/5], Step [400/600], Loss: 0.0536
Epoch [1/5], Step [500/600], Loss: 0.0354
Epoch [1/5], Step [600/600], Loss: 0.0372
Epoch [2/5], Step [100/600], Loss: 0.0216
Epoch [2/5], Step [200/600], Loss: 0.0437
Epoch [2/5], Step [300/600], Loss: 0.0578
Epoch [2/5], Step [400/600], Loss: 0.0797
Epoch [2/5], Step [500/600], Loss: 0.0350
Epoch [2/5], Step [600/600], Loss: 0.0306
Epoch [3/5], Step [100/600], Loss: 0.0564
Epoch [3/5], Step [200/600], Loss: 0.0074
Epoch [3/5], Step [300/600], Loss: 0.0058
Epoch [3/5], Step [400/600], Loss: 0.0706
Epoch [3/5], Step [500/600], Loss: 0.0485
Epoch [3/5], Step [600/600], Loss: 0.0313
Epoch [4/5], Step [100/600], Loss: 0.0287
Epoch [4/5], Step [200/600], Loss: 0.0158
Epoch [4/5], Step [300/600], Loss: 0.0355
Epoch [4/5], Step [400/600], Loss: 0.0235
Epoch [4/5], Step [500/600], Loss: 0.0076
Epoch [4/5], Step [600/600], Loss:

In [11]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98.52 %
